In [ ]:
%load_ext autoreload
%autoreload 2

# GTSAM SFM example


In [ ]:
import gtsam
import matplotlib.pyplot as plt
import numpy as np
from gtsam import symbol_shorthand

L = symbol_shorthand.L
X = symbol_shorthand.X

from gtsam.examples import SFMdata
from gtsam import (
    Cal3_S2,
    DoglegOptimizer,
    GenericProjectionFactorCal3_S2,
    Marginals,
    NonlinearFactorGraph,
    PinholeCameraCal3_S2,
    Point3,
    Pose3,
    PriorFactorPoint3,
    PriorFactorPose3,
    Rot3,
    Values,
)
from gtsam.utils import plot

In [ ]:
# Define the camera calibration parameters
K = Cal3_S2(50.0, 50.0, 0.0, 50.0, 50.0)

# Define the camera observation noise model
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)  # one pixel in u and v

# Create the set of ground-truth landmarks
points = SFMdata.createPoints()

# Create the set of ground-truth poses
poses = SFMdata.createPoses(K)

# Create a factor graph
graph = NonlinearFactorGraph()

# Add a prior on pose x1. This indirectly specifies where the origin is.
# 0.3 rad std on roll,pitch,yaw and 0.1m on x,y,z
pose_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.3, 0.1, 0.1, 0.1]))
factor = PriorFactorPose3(X(0), poses[0], pose_noise)
graph.push_back(factor)

# Simulated measurements from each camera pose, adding them to the factor graph
for i, pose in enumerate(poses):
    camera = PinholeCameraCal3_S2(pose, K)
    for j, point in enumerate(points):
        measurement = camera.project(point)
        factor = GenericProjectionFactorCal3_S2(measurement, measurement_noise, X(i), L(j), K)
        graph.push_back(factor)

In [ ]:
measurement

In [ ]:
# Because the structure-from-motion problem has a scale ambiguity, the problem is still under-constrained
# Here we add a prior on the position of the first landmark. This fixes the scale by indicating the distance
# between the first camera and the first landmark. All other landmark positions are interpreted using this scale.
point_noise = gtsam.noiseModel.Isotropic.Sigma(3, 0.1)
factor = PriorFactorPoint3(L(0), points[0], point_noise)
graph.push_back(factor)
graph.print("Factor Graph:\n")

# Create the data structure to hold the initial estimate to the solution
# Intentionally initialize the variables off from the ground truth
initial_estimate = Values()
for i, pose in enumerate(poses):
    transformed_pose = pose.retract(0.1 * np.random.randn(6, 1))
    initial_estimate.insert(X(i), transformed_pose)
for j, point in enumerate(points):
    transformed_point = point + 0.1 * np.random.randn(3)
    initial_estimate.insert(L(j), transformed_point)
initial_estimate.print("Initial Estimates:\n")

In [ ]:
# Optimize the graph and print results
params = gtsam.DoglegParams()
params.setVerbosity("TERMINATION")
optimizer = DoglegOptimizer(graph, initial_estimate, params)
print("Optimizing:")
result = optimizer.optimize()
result.print("Final results:\n")
print("initial error = {}".format(graph.error(initial_estimate)))
print("final error = {}".format(graph.error(result)))

marginals = Marginals(graph, result)
plot.plot_3d_points(1, result, marginals=marginals)
plot.plot_trajectory(1, result, marginals=marginals, scale=8)
plot.set_axes_equal(1)
plt.show()

# Fixed lag smoother


In [ ]:
import numpy as np

import gtsam
import gtsam_unstable

In [ ]:
# Define a batch fixed lag smoother, which uses
# Levenberg-Marquardt to perform the nonlinear optimization
lag = 2.0
smoother_batch = gtsam_unstable.BatchFixedLagSmoother(lag)

# Create containers to store the factors and linearization points
# that will be sent to the smoothers
new_factors = gtsam.NonlinearFactorGraph()
new_values = gtsam.Values()
new_timestamps = gtsam_unstable.FixedLagSmootherKeyTimestampMap()

# Create  a prior on the first pose, placing it at the origin
prior_mean = gtsam.Pose2(0, 0, 0)
prior_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
X1 = 0
new_factors.push_back(gtsam.PriorFactorPose2(X1, prior_mean, prior_noise))
new_values.insert(X1, prior_mean)
new_timestamps.insert((X1, 0.0))

delta_time = 0.25
time = 0.25

In [ ]:
while time <= 3.0:
    previous_key = int(1000 * (time - delta_time))
    current_key = int(1000 * time)

    # assign current key to the current timestamp
    new_timestamps.insert((current_key, time))

    # Add a guess for this pose to the new values
    # Assume that the robot moves at 2 m/s. Position is time[s] * 2[m/s]
    current_pose = gtsam.Pose2(time * 2, 0, 0)
    new_values.insert(current_key, current_pose)

    # Add odometry factors from two different sources with different error
    # stats
    odometry_measurement_1 = gtsam.Pose2(0.61, -0.08, 0.02)
    odometry_noise_1 = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.1, 0.05]))
    new_factors.push_back(
        gtsam.BetweenFactorPose2(
            previous_key, current_key, odometry_measurement_1, odometry_noise_1
        )
    )

    odometry_measurement_2 = gtsam.Pose2(0.47, 0.03, 0.01)
    odometry_noise_2 = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.05, 0.05, 0.05]))
    new_factors.push_back(
        gtsam.BetweenFactorPose2(
            previous_key, current_key, odometry_measurement_2, odometry_noise_2
        )
    )

    # Update the smoothers with the new factors. In this case,
    # one iteration must pass for Levenberg-Marquardt to accurately
    # estimate
    if time >= 0.50:
        factors_to_remove = gtsam.KeyVector([1, 2, 3])  # dummy example
        smoother_batch.update(new_factors, new_values, new_timestamps, factors_to_remove)
        print("Timestamp = " + str(time) + ", Key = " + str(current_key))
        print(smoother_batch.calculateEstimatePose2(current_key))

        new_timestamps.clear()
        new_values.clear()
        new_factors.resize(0)

    time += delta_time

# Robust / GNC optimization


In [ ]:
import gtsam

In [ ]:
params = gtsam.GncLMParams()

In [ ]:
measurement_noise = gtsam.noiseModel.Isotropic.Sigma(2, 1.0)
huber_noise = gtsam.noiseModel.Robust(gtsam.noiseModel.mEstimator.Huber(1.5), measurement_noise)

In [ ]:
optimizer = gtsam.GncLMOptimizer(graph, values, params)

# Stereo VO

- https://github.com/haidai/gtsam/blob/master/examples/StereoVOExample.cpp
- https://github.com/haidai/gtsam/blob/master/examples/StereoVOExample_large.cpp


In [ ]:
import gtsam

In [ ]:
#init(self: gtsam.gtsam.GenericStereoFactor3D, 
# measured: gtsam.gtsam.StereoPoint2, 
# noiseModel: gtsam.gtsam.noiseModel.Base, 
# poseKey: int, 
# landmarkKey: int, 
# K: gtsam.gtsam.Cal3_S2Stereo)
gtsam.GenericStereoFactor3D()  

In [ ]:
gtsam.General